In [1]:
import sys
sys.path.insert(0, '../ModeTonicEstimation/')

from fileoperations.fileoperations import getFileNamesInDir

import json
import os
import matplotlib.pyplot as plt
import copy
import numpy as np
from ModeTonicEstimation import Chordia
from ModeTonicEstimation import Evaluator as ev


In [2]:
idx = int('1') # this is the input to be taken by the cluster


In [3]:
# I/O
base_dir = '../../test_datasets/RagaRecognition_journal'
tradition_dir = 'hindustani'  # possible: makam, carnatic, hindustani
num_class_dir = '30_classes'  # possible: 5, 10, 15, 20, 25, 30

data_dir = os.path.join(base_dir, tradition_dir)

experiment_dir = os.path.join(base_dir, 'fileLists', tradition_dir, num_class_dir)
experiment_file = getFileNamesInDir(experiment_dir, keyword='*.json')[0][0]


In [4]:
experiments = json.load(open(experiment_file, 'r'))

audio_files = [os.path.join(data_dir,e[2]) for e in experiments]
pitch_files = [os.path.join(data_dir,e[2]+'.pitch') for e in experiments]
audio_mbids = [e[0] for e in experiments]
audio_tonics = [np.loadtxt(p+'.tonicFine') for p in audio_files]
audio_labels = [e[1] for e in experiments]

unique_labels = set(audio_labels)


In [5]:
# instantiate objects
evaluator = ev.Evaluator()
che = Chordia.Chordia(step_size=10, smooth_factor=15,
                      chunk_size=0, threshold=0.5, 
                      overlap=0, frame_rate=196.0/44100)


In [6]:
# divide training & testing (leave-one-out)
training_files = copy.deepcopy(audio_files)
training_pitch_files = copy.deepcopy(pitch_files)
training_mbids = copy.deepcopy(audio_mbids)
training_tonics = copy.deepcopy(audio_tonics)
training_labels = copy.deepcopy(audio_labels)

# pop the test audio from the training
test_file = training_files.pop(idx)
test_pitch_file = training_pitch_files.pop(idx)
test_mbid = training_mbids.pop(idx)
test_tonic = training_tonics.pop(idx)
test_label = training_labels.pop(idx)

model_save_dir = os.path.join(experiment_dir, 'chordia', test_mbid)


In [7]:
# Training 
models = dict()
for label in unique_labels:
    print '   Training label: ' + label
    model_file = os.path.join(model_save_dir, label+'.json')
    try:
        models[label] = che.load_collection(label, model_save_dir)
    except IOError:
        models[label] = che.train(label, training_pitch_files, training_tonics, 
                                  metric='pcd', save_dir = model_save_dir)


   Training label: 2ed9379f-14c9-49af-8e4d-f9b63e96801f
   Training label: 64e5fb9e-5569-4e80-8e6c-f543af9469c7
   Training label: 0b3bbf97-0ec3-41da-add4-722d87329ec3
   Training label: a99e07d5-20a0-467b-8dcd-aa5a095177fd
   Training label: 595de771-fcc2-414c-9df6-ae4ffd898549
   Training label: 7591faad-e68a-4550-b675-8082842c6056
   Training label: e771a74d-545d-41d5-816b-43403a818b0c
   Training label: 62b79291-73b0-4c77-a353-0f8bc6ed8362
   Training label: 48b37bed-e847-4882-8a01-5c721e07f07d
   Training label: f7fddfc0-8c1d-4dd2-90d5-5d51a99d61f8
   Training label: 3eb7ba30-4b94-432e-9618-875ee57e01ab
   Training label: 1b05a564-059f-445b-b325-cf26318367e3
   Training label: f6432fec-e9c2-4b09-9e73-c46086cbd8ea
   Training label: 54c4214c-05b9-4acc-8f77-6d5786e43a2e
   Training label: 40dbe1db-858f-4366-bef8-9076eb67340d
   Training label: 118401e7-8de8-4d81-9d8e-8070889e3fa8
   Training label: a7d98897-e2fe-4d75-b9dc-e5b4dc88e1c6
   Training label: 6f13484e-6fdd-402d-baf3-38358

In [16]:
# Testing 
print 'Testing audio: ' + test_mbid
res = che.estimate(test_pitch_file, mode_names=list(unique_labels), 
                   mode_dir=model_save_dir, est_mode=True, 
                   distance_method='bhat',
                   metric='pcd', tonic_freq=test_tonic)[0]

print res == test_label

Testing audio: 9117fa28-df28-4bb8-a0e3-b70e60fed262
True


In [ ]:
# # get the data into appropriate format
# [pitch_paths, pitch_base, pitch_fname] = fo.getFileNamesInDir(data_dir, '.pitch')
# tonic_paths = [os.path.splitext(p)[0] + '.tonic' for p in pitch_paths]
# mode_labels = []
# for p in pitch_base:
#     for r in modes:
#         if r in p:
#             mode_labels.append(r)
            

In [ ]:
# # make the data a single dictionary for housekeeping
# data = []
# for p, f, t, r in zip(pitch_paths, pitch_fname, tonic_paths, mode_labels):
#     data.append({'file':p, 'name':os.path.splitext(f)[0],
#                'tonic':float(np.loadtxt(t)), 'mode':r})


In [ ]:
# # experiments
# results = dict()
# for key, fold in folds.iteritems():
#     # Training 
#     print key
#     models = dict()
#     for cur_mode in modes:
#         [file_list, tonic_list] = zip(*[(rec['file'], rec['tonic']) for rec in fold['train']
#                                         if rec['mode'] == cur_mode])
#         models[cur_mode] = che.train(cur_mode, file_list, tonic_list, metric='pcd', 
#                                      save_dir = os.path.join(train_savefolder, key))
                                     
#     # Raag Recognition
#     results[key] = []
#     for rec in fold['test']:
#         rec['pitch'] = np.loadtxt(rec['file'])[:,1]
#         res = che.estimate(rec['pitch'], mode_names=modes, 
#                            mode_dir=os.path.join(train_savefolder, key), 
#                            est_tonic=False, est_mode=True, distance_method='bhat',
#                            metric='pcd', ref_freq=rec['tonic'])[0]

#         # evaluate
#         results[key].append(evaluator.mode_evaluate(rec['file'], res, rec['mode']))

#     print key + ": " + str(100*np.mean([r['mode_eval'] for r in results[key]])) + '%'
    